# Подключение библиотек.

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import tensorflow as tf
import re
import string
from sklearn.cluster import KMeans


In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore')

In [3]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

# Загрузка сохраненной модели

In [29]:
reconstructed_model_two = keras.models.load_model("best_model_2.h5")

In [31]:
reconstructed_model = keras.models.load_model("best_model.h5")

In [32]:
result = reconstructed_model.evaluate(X_test, Y_test, batch_size=128)

3/3 [==============================] - 2s 32ms/step - loss: 0.1390 - accuracy: 0.7855


In [33]:
result

[0.13896004855632782, 0.785515308380127]

In [127]:
comm = '''Всем доброго времени суток и хорошего настроения'''

In [128]:
predictions = reconstructed_model.predict(tokenizer.texts_to_sequences([comm]))

KeyError: 'pop from an empty set'

In [132]:
predictions

array([[0.95889664]], dtype=float32)

In [133]:
display(round(predictions[0][0]))

1

Инициализация функции предобработки текста

In [6]:
def preprocess_tweet(tweet):
  tweet = tweet.lower() # lowercase

  tweet = re.sub(r"http\S+|https\S+","", tweet, flags=re.MULTILINE) #rem url

  tweet = tweet.translate(str.maketrans("", "", string.punctuation)) #punctuation

  tweet = re.sub(r"\@\w+|\#", "", tweet) # user ref and #

  #stop_Wrd
  tweet_tokens = word_tokenize(tweet)
  filtered_words = [word for word in tweet_tokens if word not in stop_words]
  

  #stem
  ps = PorterStemmer()
  stemmed_words = [ps.stem(w) for w in filtered_words]

  #lemma
  lemmatizer = WordNetLemmatizer()
  lemma_words = [lemmatizer.lemmatize(w,pos='a') for w in stemmed_words] 

  return " ".join(lemma_words)


Загрузка обучающего и тестового датасета

In [8]:
train = pd.read_csv("training.csv",
                        encoding="latin-1",
                        names=['sentiment', 'num', "date", 'query','autor','text'])
train

,sentiment,num,date,query,autor,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [9]:
test= pd.read_csv("testdata.csv",
                        encoding="latin-1",
                        names=['sentiment', 'num', "date", 'query','autor','text'])
test

,sentiment,num,date,query,autor,text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
...,...,...,...,...,...,...
493,2,14072,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...
494,0,14073,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat..."
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [9]:
train = train[['text','sentiment']]
train


,text,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0
...,...,...
1599995,Just woke up. Having no school is the best fee...,4
1599996,TheWDB.com - Very cool to hear old Walt interv...,4
1599997,Are you ready for your MoJo Makeover? Ask me f...,4
1599998,Happy 38th Birthday to my boo of alll time!!! ...,4


In [11]:
test = test[['text','sentiment']]
test


,text,sentiment
0,@stellargirl I loooooooovvvvvveee my Kindle2. ...,4
1,Reading my kindle2... Love it... Lee childs i...,4
2,"Ok, first assesment of the #kindle2 ...it fuck...",4
3,@kenburbary You'll love your Kindle2. I've had...,4
4,@mikefish Fair enough. But i have the Kindle2...,4
...,...,...
493,Ask Programming: LaTeX or InDesign?: submitted...,2
494,"On that note, I hate Word. I hate Pages. I hat...",0
495,Ahhh... back in a *real* text editing environm...,4
496,"Trouble in Iran, I see. Hmm. Iran. Iran so far...",0


Предобработка данных

In [123]:
data = [preprocess_tweet(t) for t in train['text']]


In [12]:
data_test = [preprocess_tweet(t) for t in test['text']]

In [125]:
data_pd = pd.DataFrame(data)
data_pd.head(3)

,0
0,switchfoot awww that bummer shoulda got david ...
1,upset cant updat facebook text might cri resul...
2,kenichan dive mani time ball manag save 50 res...


In [13]:
data_test_pd = pd.DataFrame(data_test)
data_test_pd

,0
0,stellargirl loooooooovvvvvvee kindle2 dx cool ...
1,read kindle2 love lee child good read
2,ok first asses kindle2 fuck rock
3,kenburbari youll love kindle2 ive mine month n...
4,mikefish fair enough kindle2 think perfect
...,...
493,ask program latex indesign submit calcio1 link...
494,note hate word hate page hate latex said hate ...
495,ahhh back real text edit environ lt3 latex
496,troubl iran see hmm iran iran far away flockof...


In [14]:
train["text"] = data_pd

NameError: name 'data_pd' is not defined

In [130]:
train

,text,sentiment
0,switchfoot awww that bummer shoulda got david ...,0
1,upset cant updat facebook text might cri resul...,0
2,kenichan dive mani time ball manag save 50 res...,0
3,whole bodi feel itchi like fire,0
4,nationwideclass behav im mad cant see,0
...,...,...
1599995,woke school best feel ever,4
1599996,thewdbcom cool hear old walt interview â «,4
1599997,readi mojo makeov ask detail,4
1599998,happi 38th birthday boo alll time tupac amaru ...,4


In [15]:
test["text"] = data_test_pd

In [20]:
test.head(8)

,text,sentiment
0,stellargirl loooooooovvvvvvee kindle2 dx cool ...,4
1,read kindle2 love lee child good read,4
2,ok first asses kindle2 fuck rock,4
3,kenburbari youll love kindle2 ive mine month n...,4
4,mikefish fair enough kindle2 think perfect,4
5,richardebak big im quit happi kindle2,4
6,fuck economi hate aig non loan given ass,0
7,jqueri new best friend,4


Сохранение обработанных данных, для исключения повторных операций с предобработкой

In [132]:
train[['text',"sentiment"]].to_csv("Train.csv")

In [119]:
test[['text',"sentiment"]].to_csv("test.csv")

# Чтение сохраненных данных

In [7]:
train_140 = pd.read_csv("Train.csv", index_col="id").astype(str)
train_140

,text,sentiment
id,,
0,switchfoot awww that bummer shoulda got david ...,0
1,upset cant updat facebook text might cri resul...,0
2,kenichan dive mani time ball manag save 50 res...,0
3,whole bodi feel itchi like fire,0
4,nationwideclass behav im mad cant see,0
...,...,...
1599995,woke school best feel ever,4
1599996,thewdbcom cool hear old walt interview â «,4
1599997,readi mojo makeov ask detail,4


In [8]:
test_140 = pd.read_csv("test.csv", index_col="id").astype(str)
test_140


,text,sentiment
id,,
0,stellargirl loooooooovvvvvvee kindle2 dx cool ...,4
1,read kindle2 love lee child good read,4
2,ok first asses kindle2 fuck rock,4
3,kenburbari youll love kindle2 ive mine month n...,4
4,mikefish fair enough kindle2 think perfect,4
...,...,...
493,ask program latex indesign submit calcio1 link...,2
494,note hate word hate page hate latex said hate ...,0
495,ahhh back real text edit environ lt3 latex,4


# Приведение к int

In [9]:
train_140.sentiment = train_140.sentiment.astype(int)
train_140.sentiment[0]

0

In [10]:
test_140.sentiment = test_140.sentiment.astype(int)
test_140.sentiment[0]

4

# Токенизация текста

In [11]:
tokenizer = Tokenizer()

In [12]:
tokenizer.fit_on_texts(train_140.text)

In [30]:
tokenizer.word_index

{'im': 1,
 'go': 2,
 'good': 3,
 'get': 4,
 'day': 5,
 'work': 6,
 'like': 7,
 'love': 8,
 'got': 9,
 'dont': 10,
 'today': 11,
 'time': 12,
 'cant': 13,
 'thank': 14,
 'want': 15,
 'back': 16,
 'miss': 17,
 'one': 18,
 'lol': 19,
 'know': 20,
 'u': 21,
 'see': 22,
 'feel': 23,
 'think': 24,
 'realli': 25,
 'well': 26,
 'hope': 27,
 'night': 28,
 'watch': 29,
 'need': 30,
 'still': 31,
 'make': 32,
 'new': 33,
 'na': 34,
 'amp': 35,
 'home': 36,
 'look': 37,
 'come': 38,
 'oh': 39,
 '2': 40,
 'much': 41,
 'last': 42,
 'twitter': 43,
 'morn': 44,
 'great': 45,
 'tomorrow': 46,
 'wish': 47,
 'wait': 48,
 'ill': 49,
 'sleep': 50,
 'that': 51,
 'haha': 52,
 'bad': 53,
 'sad': 54,
 'fun': 55,
 'tri': 56,
 'right': 57,
 'week': 58,
 'follow': 59,
 'happi': 60,
 'didnt': 61,
 'would': 62,
 'thing': 63,
 'sorri': 64,
 'tonight': 65,
 'say': 66,
 'friend': 67,
 'way': 68,
 'take': 69,
 'nice': 70,
 'gon': 71,
 'though': 72,
 'ive': 73,
 'hate': 74,
 'even': 75,
 'late': 76,
 'start': 77,
 'bed'

In [13]:
sequence = tokenizer.texts_to_sequences(train_140.text)


In [32]:
print(train_140.text[10])
print(sequence[10])

spring break plain citi snow
[1318, 341, 2666, 485, 1445]


In [61]:
tokenizer.word_index[['spring','fuck']]

TypeError: unhashable type: 'list'

In [35]:
display(sequence)

[[15741, 380, 51, 1080, 3087, 9, 718, 7673, 1676, 5],
 [604, 13, 226, 448, 366, 207, 240, 983, 83, 11, 192, 1075],
 [22657, 3704, 227, 12, 883, 705, 513, 1163, 355, 2, 2839],
 [338, 665, 23, 2574, 7, 894],
 [29697, 4138, 1, 464, 13, 22],
 [228717, 338, 1952],
 [30, 396],
 [134611, 91, 87, 12, 22, 96, 101, 176, 176, 19, 1, 426, 14, 679],
 [41394, 681, 61],
 [27842, 2145, 228718],
 [1318, 341, 2666, 485, 1445],
 [18700, 791],
 [52924, 321, 1330, 29, 155, 13439, 1390, 2855],
 [228719, 774, 739, 386, 98, 126, 320],
 [228720, 2020, 105, 61, 2395, 25, 72, 3601, 31805, 228721],
 [228722, 47, 9, 29, 17, 228723, 2104],
 [2416, 937, 1618, 139, 1470, 29, 648, 31806, 4580, 414],
 [1258, 2307],
 [96144, 594, 73, 125, 15, 22, 1552, 8, 3216],
 [134612, 39, 504, 275, 2351, 1482, 275],
 [228724, 5, 61, 4, 41, 123],
 [18, 67, 116, 296, 219, 3297, 3221, 7386, 73, 12, 526],
 [228725, 1281, 640, 571],
 [58, 2, 27],
 [41395, 289, 465, 46],
 [74, 116, 317, 82],
 [2, 240, 50, 29, 3063],
 [1, 54, 228726],
 [28

In [15]:
len_text = max([len(n) for n in sequence])
len_text


56

In [16]:
X_train = pad_sequences(sequence,maxlen=len_text)
X_train 


array([[     0,      0,      0, ...,   7673,   1676,      5],
       [     0,      0,      0, ...,     11,    192,   1075],
       [     0,      0,      0, ...,    355,      2,   2839],
       ...,
       [     0,      0,      0, ...,   6417,    296,   1545],
       [     0,      0,      0, ...,  10438,  90209,  71955],
       [     0,      0,      0, ..., 225968, 725606, 725607]])

In [17]:
Y_train = train_140[train_140.sentiment == 0].sentiment
Y_train2 = train_140[train_140.sentiment == 4].sentiment - 3
Y_train = Y_train.append(Y_train2)
Y_train #= Y_train.to_numpy().reshape(-1,1)

id
0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: sentiment, Length: 1600000, dtype: int32

In [18]:
test_140.head(11)

,text,sentiment
id,,
0,stellargirl loooooooovvvvvvee kindle2 dx cool ...,4
1,read kindle2 love lee child good read,4
2,ok first asses kindle2 fuck rock,4
3,kenburbari youll love kindle2 ive mine month n...,4
4,mikefish fair enough kindle2 think perfect,4
5,richardebak big im quit happi kindle2,4
6,fuck economi hate aig non loan given ass,0
7,jqueri new best friend,4
8,love twitter,4


In [19]:
gr_1 = test_140[test_140.sentiment == 0][["text", "sentiment"]]
display(gr_1)
gr_2 = test_140[test_140.sentiment == 4][["text", "sentiment"]]
display(gr_2)
gr_2 = gr_2.append(gr_1)
display(gr_2)

,text,sentiment
id,,
6,fuck economi hate aig non loan given ass,0
11,karoli firmli believ obamapelosi zero desir ci...,0
14,dear nike stop flywir shit wast scienc ugli lo...,0
16,talk guy last night tell die hard spur fan als...,0
18,ludajuic lebron beast im still cheer 4 atil end,0
...,...,...
485,monday alreadi iran may implod kitchen disast ...,0
490,creat first latex file scratch didnt work well...,0
494,note hate word hate page hate latex said hate ...,0


,text,sentiment
id,,
0,stellargirl loooooooovvvvvvee kindle2 dx cool ...,4
1,read kindle2 love lee child good read,4
2,ok first asses kindle2 fuck rock,4
3,kenburbari youll love kindle2 ive mine month n...,4
4,mikefish fair enough kindle2 think perfect,4
...,...,...
487,get readi test burger receip weekend bobbi fla...,4
489,lam love bobbi flay favorit rt terrysimpson bf...,4
491,use linux love much nice window look forward u...,4


,text,sentiment
id,,
0,stellargirl loooooooovvvvvvee kindle2 dx cool ...,4
1,read kindle2 love lee child good read,4
2,ok first asses kindle2 fuck rock,4
3,kenburbari youll love kindle2 ive mine month n...,4
4,mikefish fair enough kindle2 think perfect,4
...,...,...
485,monday alreadi iran may implod kitchen disast ...,0
490,creat first latex file scratch didnt work well...,0
494,note hate word hate page hate latex said hate ...,0


In [20]:
gr_2.loc[(gr_2.sentiment==4), ('sentiment')] = 1
gr_2

,text,sentiment
id,,
0,stellargirl loooooooovvvvvvee kindle2 dx cool ...,1
1,read kindle2 love lee child good read,1
2,ok first asses kindle2 fuck rock,1
3,kenburbari youll love kindle2 ive mine month n...,1
4,mikefish fair enough kindle2 think perfect,1
...,...,...
485,monday alreadi iran may implod kitchen disast ...,0
490,creat first latex file scratch didnt work well...,0
494,note hate word hate page hate latex said hate ...,0


In [21]:
sequence_test = tokenizer.texts_to_sequences(gr_2.text)

In [22]:
X_test = pad_sequences(sequence_test,maxlen = len_text)
text = len(X_test[0])
text

56

In [23]:
Y_test = gr_2.sentiment


Проверка размерности

In [24]:
print(len(Y_test),len(X_test),len(X_train),len(Y_train))

359 359 1600000 1600000


# Создание модели

In [77]:
model = tf.keras.Sequential()

In [78]:
model = Sequential()
model.add(layers.Embedding(1000, 128, input_length=len_text))
model.add(layers.LSTM(192,return_sequences=True))
model.add(layers.LSTM(320,return_sequences=True))
model.add(layers.LSTM(32,return_sequences=True))
model.add(layers.LSTM(480))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(1,"relu"))

In [83]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [80]:
model.compile(loss='mse',
              optimizer='adam',
              metrics = ['accuracy'])


In [84]:
model_save_path = "best_model_2.h5"
checkpoint_callback = ModelCheckpoint(model_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [82]:
history = model.fit(X_train,
                    Y_train,
                    batch_size=168,
                    epochs=20,
                    callbacks=[checkpoint_callback],
                    validation_split=0.1)

Epoch 1/20
8572/8572 [==============================] - ETA: 0s - loss: 0.1624 - accuracy: 0.7578
Epoch 1: val_accuracy improved from -inf to 0.59653, saving model to best_model.h5
8572/8572 [==============================] - 851s 99ms/step - loss: 0.1624 - accuracy: 0.7578 - val_loss: 0.2474 - val_accuracy: 0.5965
Epoch 2/20
8572/8572 [==============================] - ETA: 0s - loss: 0.1545 - accuracy: 0.7715
Epoch 2: val_accuracy improved from 0.59653 to 0.71978, saving model to best_model.h5
8572/8572 [==============================] - 846s 99ms/step - loss: 0.1545 - accuracy: 0.7715 - val_loss: 0.1960 - val_accuracy: 0.7198
Epoch 3/20
8572/8572 [==============================] - ETA: 0s - loss: 0.1509 - accuracy: 0.7778
Epoch 3: val_accuracy did not improve from 0.71978
8572/8572 [==============================] - 847s 99ms/step - loss: 0.1509 - accuracy: 0.7778 - val_loss: 0.1950 - val_accuracy: 0.7194
Epoch 4/20
8572/8572 [==============================] - ETA: 0s - loss: 0.1480

In [85]:
history_copy = model.fit(X_train,
                    Y_train,
                    batch_size=168,
                    epochs=20,
                    callbacks=[checkpoint_callback],
                    validation_split=0.1)

Epoch 1/20
8571/8572 [============================>.] - ETA: 0s - loss: 0.3475 - accuracy: 0.8652
Epoch 1: val_accuracy improved from -inf to 0.71666, saving model to best_model_2.h5
8572/8572 [==============================] - 859s 99ms/step - loss: 0.3475 - accuracy: 0.8652 - val_loss: 0.6680 - val_accuracy: 0.7167
Epoch 2/20
8572/8572 [==============================] - ETA: 0s - loss: 0.3559 - accuracy: 0.8613
Epoch 2: val_accuracy did not improve from 0.71666
8572/8572 [==============================] - 855s 100ms/step - loss: 0.3559 - accuracy: 0.8613 - val_loss: 0.5915 - val_accuracy: 0.7155
Epoch 3/20
8572/8572 [==============================] - ETA: 0s - loss: 0.3642 - accuracy: 0.8578
Epoch 3: val_accuracy did not improve from 0.71666
8572/8572 [==============================] - 856s 100ms/step - loss: 0.3642 - accuracy: 0.8578 - val_loss: 0.6443 - val_accuracy: 0.7131
Epoch 4/20
8572/8572 [==============================] - ETA: 0s - loss: 0.3548 - accuracy: 0.8587
Epoch 4: va

# Использование библиотеки Keras - tuner для подбора оптимальных параметров модели

In [64]:
import kerastuner as kt

In [66]:
def buld_model(hp):
    model = Sequential()
    model.add(layers.Embedding(1000, 128, input_length=len_text))
    model.add(layers.LSTM(64))
    model.add(layers.Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model
    

In [68]:
def build_model(hp):
    model = Sequential()
    model.add(layers.Embedding(1000, 128, input_length=len_text))
    model.add(layers.LSTM(192,return_sequences=True))
    model.add(layers.LSTM(320,return_sequences=True))
    model.add(layers.LSTM(32,return_sequences=True))
    model.add(layers.LSTM(480))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1,"relu"))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['accuracy'])
    return model

In [69]:
tuner= kt.BayesianOptimization(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        directory="test_modell"
        )

INFO:tensorflow:Reloading Oracle from existing project test_modell\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from test_modell\untitled_project\tuner0.json


In [70]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
input_unit (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
lstm_0_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
layer_2_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
Dropout_rate (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
lstm_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
lstm_2_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
l

In [262]:
Y_test = XY_test.sentiment

In [ ]:
tuner.search(
        x=X_train,
        y=Y_train,
        epochs=20,
        batch_size=128,
        validation_data=(X_test,Y_test)
) 

Trial 5 Complete [04h 59m 03s]
val_accuracy: 0.7994428873062134

Best val_accuracy So Far: 0.8161559700965881
Total elapsed time: 1d 07h 10m 04s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
512               |192               |input_unit
1                 |2                 |n_layers
288               |320               |lstm_0_units
256               |480               |layer_2_neurons
0.5               |0.4               |Dropout_rate
relu              |relu              |dense_activation
32                |32                |lstm_1_units
32                |None              |lstm_2_units
32                |None              |lstm_3_units

Epoch 1/20
12500/12500 [==============================] - 1032s 82ms/step - loss: 0.1638 - accuracy: 0.7585 - val_loss: 0.1439 - val_accuracy: 0.7939
Epoch 2/20
12500/12500 [==============================] - 1026s 82ms/step - loss: 0.1550 - accuracy: 0.7724 - val_loss: 0.1408 - val_accuracy: 0.7744
Epoch 3/20
125

In [72]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [75]:
model = tuner.hypermodel.build(best_hps)

In [76]:
history = model.fit(X_train,
                    Y_train,
                    batch_size=168,
                    epochs=20,
                    callbacks=[checkpoint_callback],
                    validation_split=0.1)

Epoch 1/20
2057/8572 [======>.......................] - ETA: 10:08 - loss: 0.1733 - accuracy: 0.7388

KeyboardInterrupt: 

In [36]:
#Kmean

In [39]:
clust = KMeans().fit(X_train,Y_train)

In [63]:
X_test

array([[     0,      0,      0, ...,   7512,   2802,     57],
       [     0,      0,      0, ...,   2440,      3,    140],
       [     0,      0,      0, ...,  57527,  99758, 172152],
       ...,
       [     0,      0,      0, ...,   1490,    122,  15037],
       [     0,      0,      0, ...,   6324,    273,    161],
       [     0,      0,      0, ...,     63,  20061,     54]])

In [64]:
Y_test

0      4
1      4
2      4
3      4
4      4
      ..
493    2
494    0
495    4
496    0
497    0
Name: sentiment, Length: 498, dtype: int64